In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# ---------- reciprocal lattice ----------
def reciprocal_vectors(a):
    """Generates the 3 shortest moiré reciprocal vectors G_1,2,3, 60° apart, six-fold symmetry"""
    g = 4 * np.pi / (np.sqrt(3) * a)  # (length |G|=4π/√3a)
    G1 = np.array([g, 0.0])  # first vector along x-axis

    def rot(v, ang):
        return np.array(
            [
                np.cos(ang) * v[0] - np.sin(ang) * v[1],  # rotate G1 by +60°, get G2
                np.sin(ang) * v[0] + np.cos(ang) * v[1],  # rotate G1 by –60°, get G3
            ]
        )

    return np.stack([G1, rot(G1, np.pi / 3), rot(G1, -np.pi / 3)])

In [3]:
a_m = 8.5
R = torch.randn(7, 2)
G_vectors = torch.from_numpy(reciprocal_vectors(a_m)).float()
print(G_vectors)
G1_T = G_vectors[0].unsqueeze(-1)
G2_T = G_vectors[1].unsqueeze(-1)
G1_R = torch.matmul(R, G1_T)
G2_R = torch.matmul(R, G2_T)
features_R = torch.cat(
    (torch.sin(G1_R), torch.sin(G2_R), torch.cos(G1_R), torch.cos(G2_R)), dim=1
)
print(features_R.shape)

tensor([[ 0.8536,  0.0000],
        [ 0.4268,  0.7392],
        [ 0.4268, -0.7392]])
torch.Size([7, 4])


In [4]:
class FeedForwardLayer(nn.Module):
    def __init__(self, L: int) -> None:
        super().__init__()

        # W^(l+1) h^l + b^(l+1)
        self.Wl_1p = nn.Linear(L, L)

        # (nonlinear) hyperbolic tangent activation function
        self.tanh = nn.Tanh()

    def forward(self, hl: torch.Tensor) -> torch.Tensor:
        # h^l + tanh( W^(l+1) h^l + b^(l+1) )
        return hl + self.tanh(self.Wl_1p(hl))

In [5]:
class SlaterNet(nn.Module):
    def __init__(self, a: float, N: int, L: int = 4, num_layers: int = 3) -> None:
        super().__init__()

        # first, get G vectors
        G_vectors = torch.from_numpy(reciprocal_vectors(a)).float()
        self.G1_T = G_vectors[0].unsqueeze(-1)
        self.G2_T = G_vectors[1].unsqueeze(-1)

        # input embedding matrix
        self.W_0 = nn.Linear(4, L, bias=False)

        # multilayer perceptron neural network layers
        self.MLP_layers = nn.ModuleList(
            [FeedForwardLayer(L) for _ in range(num_layers)]
        )

    def forward(self, R: torch.Tensor) -> torch.Tensor:
        # R should be of shape (N, 2)
        # first we need to compute the periodic features
        G1_R = torch.matmul(R, self.G1_T)
        G2_R = torch.matmul(R, self.G2_T)
        features_R = torch.cat(
            (torch.sin(G1_R), torch.sin(G2_R), torch.cos(G1_R), torch.cos(G2_R)), dim=1
        )
        # shape should now be (N, 4)

        # embed in higher_dimensional space to get h^0
        h0_R = self.W_0(features_R)

        pass